# Assignment 4 - Problem 2

## Modeling Frog Escape Problem as Finite MDP (Code from last assignment)

For the *frog-escape problem*, we have the state space $S = \{0, 1, ..., n\}$, the terminal states $T = {0, n}$, and the action space $A = {A, B}$ (croaking). 

We have the following transition probabilities $\mathcal{P}(s,a,s') \text{ for all } i \in N$:

$$\mathcal{P}(i,A,i-1) = \frac{i}{n}$$
$$\mathcal{P}(i,A,i+1) = \frac{n-1}{i}$$
$$\mathcal{P}(i,B, s') = \frac{1}{n} \text{ for all }s' \in \{0,...,i-1,i+1,...n\}$$

All other transition probabilities are 0. Finally, we set the reward function $\mathcal{R_T}(s,a,s')$ to be:

$$\mathcal{R_T}(i,A,n) = 1 \text{ for all } i \in N $$
$$\mathcal{R_T}(i,B,n) = 1 \text{ for all } i \in N $$

Rewards for landing all other states are 0. Then, the only way to get any reward is by escaping (landing on n), and the Optimal Value Function will represent the probability of escaping the bond.

In [1]:
from dataclasses import dataclass
from typing import Tuple, Dict
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.markov_decision_process import FinitePolicy, StateActionMapping
from rl.markov_process import FiniteMarkovProcess, FiniteMarkovRewardProcess
from rl.distribution import Categorical, Constant
from scipy.stats import poisson
import itertools
from pprint import pprint


In [2]:
@dataclass(frozen=True)
class PadState:
    pad_num: int
        
PadTransMapping = StateActionMapping[PadState, int]

In [3]:
class FrogEscMDP(FiniteMarkovDecisionProcess[PadState, int]):

    def __init__(self, n: int):
        self.n: int = n     
        super().__init__(self.get_action_transition_reward_map())

    def get_action_transition_reward_map(self) -> PadTransMapping:
        #state -> action -> state-reward -> p 
        d: Dict[PadState, Dict[str, Categorical[Tuple[PadState, float]]]] = {}
        #define rewards
        rewards: Dict[int, float] = {s: 0 for s in range(n)}
        rewards[n] = 1
            
        for state in range(1, self.n): #non-terminal states
            #action -> state-reward -> p
            d1: Dict[str, Categorical[Tuple[PadState, float]]] = {}
            
            #croaks A
            #state-reward -> p (to be turned into Categorical)
            sr_probs_dict_A: Dict[Tuple[PadState, float], float] = \
                {(PadState(state-1), rewards[state-1]): state/n, \
                (PadState(state+1), rewards[state+1]): (n-state)/n}
            d1['A'] = Categorical(sr_probs_dict_A)
            
            #croaks B
            sr_probs_dict_B: Dict[Tuple[PadState, float], float] = \
                {(PadState(next_state), rewards[next_state]): 1/n for next_state in range(n+1) if n != state}          
            d1['B'] = Categorical(sr_probs_dict_B)

            d[PadState(state)] = d1
            
        #terminal states
        d[PadState(0)] = None
        d[PadState(n)] = None
        return d

In [4]:
# build model
n = 6
frog_mdp: FiniteMarkovDecisionProcess[PadState, int] =\
    FrogEscMDP(n=n)

# print("MDP Transition Map")
# print("------------------")
# print(frog_mdp)

In [8]:
from rl.dynamic_programming import policy_iteration_result
from rl.dynamic_programming import value_iteration_result

import time

In [33]:
type(frog_mdp.mapping[PadState(1)]['A'])

rl.distribution.Categorical

In [35]:
frog_mdp.mapping[PadState(1)]['A']

{(PadState(pad_num=0), 0): 0.16666666666666666, (PadState(pad_num=2), 0): 0.8333333333333334}

In [32]:
for x,p in frog_mdp.mapping[PadState(1)]['A']:
    print(x)
    print(p)

(PadState(pad_num=0), 0)
0.16666666666666666
(PadState(pad_num=2), 0)
0.8333333333333334


In [25]:
frog_mdp.mapping[PadState(1)]

{'A': {(PadState(pad_num=0), 0): 0.16666666666666666, (PadState(pad_num=2), 0): 0.8333333333333334},
 'B': {(PadState(pad_num=0), 0): 0.14285714285714288, (PadState(pad_num=1), 0): 0.14285714285714288, (PadState(pad_num=2), 0): 0.14285714285714288, (PadState(pad_num=3), 0): 0.14285714285714288, (PadState(pad_num=4), 0): 0.14285714285714288, (PadState(pad_num=5), 0): 0.14285714285714288, (PadState(pad_num=6), 1): 0.14285714285714288}}

In [19]:
frog_mdp.mapping

{PadState(pad_num=1): {'A': {(PadState(pad_num=0), 0): 0.16666666666666666, (PadState(pad_num=2), 0): 0.8333333333333334},
  'B': {(PadState(pad_num=0), 0): 0.14285714285714288, (PadState(pad_num=1), 0): 0.14285714285714288, (PadState(pad_num=2), 0): 0.14285714285714288, (PadState(pad_num=3), 0): 0.14285714285714288, (PadState(pad_num=4), 0): 0.14285714285714288, (PadState(pad_num=5), 0): 0.14285714285714288, (PadState(pad_num=6), 1): 0.14285714285714288}},
 PadState(pad_num=2): {'A': {(PadState(pad_num=1), 0): 0.3333333333333333, (PadState(pad_num=3), 0): 0.6666666666666666},
  'B': {(PadState(pad_num=0), 0): 0.14285714285714288, (PadState(pad_num=1), 0): 0.14285714285714288, (PadState(pad_num=2), 0): 0.14285714285714288, (PadState(pad_num=3), 0): 0.14285714285714288, (PadState(pad_num=4), 0): 0.14285714285714288, (PadState(pad_num=5), 0): 0.14285714285714288, (PadState(pad_num=6), 1): 0.14285714285714288}},
 PadState(pad_num=3): {'A': {(PadState(pad_num=2), 0): 0.5, (PadState(pad_num

In [24]:
(frog_mdp.mapping[PadState(2)]['A']).expectation(lambda s_r: s_r[1])

0.0

## Using functions policy_iteration and value_iteration 


In [12]:
print("MDP Policy Iteration Optimal Value Function and Optimal Policy")
print("--------------")
t1 = time.time()
opt_vf_pi, opt_policy_pi = policy_iteration_result(
    frog_mdp,
    gamma=1.0
)
t2 = time.time()
pprint(opt_vf_pi)
print(opt_policy_pi)
print(f"It took {(t2-t1)*1000} ms for policy iteration")
print()

print("MDP Value Iteration Optimal Value Function and Optimal Policy")
print("--------------")
t3 = time.time()
opt_vf_vi, opt_policy_vi = value_iteration_result(frog_mdp, gamma=1.0)
t4 = time.time()
pprint(opt_vf_vi)
print(opt_policy_vi)
print(f"It took {(t4-t3)*1000} ms for value iteration")
print()


MDP Policy Iteration Optimal Value Function and Optimal Policy
--------------
{PadState(pad_num=4): 0.7444417962236649,
 PadState(pad_num=5): 0.7870264522069902,
 PadState(pad_num=1): 0.6594154577345575,
 PadState(pad_num=2): 0.7019054789306673,
 PadState(pad_num=3): 0.7231639388837591}
For State PadState(pad_num=1):
  Do Action B with Probability 1.000
For State PadState(pad_num=2):
  Do Action A with Probability 1.000
For State PadState(pad_num=3):
  Do Action A with Probability 1.000
For State PadState(pad_num=4):
  Do Action A with Probability 1.000
For State PadState(pad_num=5):
  Do Action A with Probability 1.000

It took 53.347110748291016 ms for policy iteration

MDP Value Iteration Optimal Value Function and Optimal Policy
--------------
{PadState(pad_num=4): 0.7444351834022599,
 PadState(pad_num=5): 0.7870207101314249,
 PadState(pad_num=1): 0.6594110592638028,
 PadState(pad_num=2): 0.7018993329449162,
 PadState(pad_num=3): 0.7231572915541348}
For State PadState(pad_num=1):
 

Value iteration is much faster as we had expected since it performs partial policy evaluation per iteration